In [2]:
%display latex

<h1><p style="color:#16A085"> Álgebra Computacional </p style></h1>
<h2> Índice </h2>

- 1. Algoritmo de Euclides
    - 1.1 Algoritmo de Euclides para un Dominio Euclídeo
    - 1.2 Algoritmo de Euclides extendido para un Dominio Euclídeo
    - 1.3 Ejemplos en $\mathbb{Z}$
    - 1.4 Ejemplos en $\mathbb{Q}[x]$
    - 1.5 Ejemplos en $\mathbb{R}[x]$
    - 1.6 Ejemplos en $\mathbb{Z}[i]$
    

# 1. Algoritmo de Euclides

# 1.1 Algoritmo de Euclides para un Dominio Euclídeo

In [1]:
def euclides(a, b, mod):
    if (b == 0):
        return a
    else:
        return euclides (b, (mod(a, b)), mod)

## 1.2 Algoritmo de Euclides extendido para un Dominio Euclídeo

In [2]:
def extendedEuclides(a, b, div_mod):
    if (b == 0):
        return a, 1, 0 
    else:
        q, r = div_mod(a, b)
        gcd, d, e = extendedEuclides(b, r, div_mod)
        return gcd, e, (d -q * e)

## 1.3 Ejemplos en $\mathbb{Z}$

In [3]:
def mod_int(a, b): return a % b
def div_mod_int(a, b): return a // b, a % b

print(euclides(55, 22, mod_int))
print(extendedEuclides(55, 22, div_mod_int))

11
(11, 1, -2)


## 1.4 Ejemplos en $\mathbb{Q}[x]$

In [4]:
def div_mod_pol(a, b): return a.quo_rem(b);
def mod_pol(a, b): _, r =  div_mod_pol(a, b); return r
    
R.<x> = QQ[]

a = (x^2 +2*x + 1)
b =  (x^2 - 1)

print(euclides(a, b, mod_pol))
print(extendedEuclides(a, b, div_mod_pol))

c = (x+2/3)*(x-3/2)*(x^2+1)
d = (x+2/3)*(x+2/3)*(x^2+1)
print(euclides(c, d, mod_pol)).monic()

2*x + 2
(2*x + 2, 1, -1)
x^3 + 2/3*x^2 + x + 2/3


## 1.5 Ejemplos en $\mathbb{R}[x]$

In [5]:
R.<x> = RR[]
a = (x^2 +2*x + 1)
b = (x^2 - 1)
print(euclides(a, b, mod_pol))
print(extendedEuclides(a, b, div_mod_pol))

c = (x+2/3)*(x-2/3)*(x^2+1)
d = (x+2/3)*(x+2/3)*(x^2+1)
print(euclides(c, d, mod_pol))

2.00000000000000*x + 2.00000000000000
(2.00000000000000*x + 2.00000000000000, 1.00000000000000, -1.00000000000000)
-3.28954970259306e-17


## 1.6 Ejemplos en $\mathbb{Z}[i]$

In [6]:
def divide_gaussian_integers(z, d):
    r,q = z, 0
    s = 1 if norm(r - d) < norm(r) else -1
    while norm(r - s*d) < norm(r):
        q = q + s
        r = r - s*d

    s = I if norm(r - I*d) < norm(r) else -I
    while norm(r - s*d) < norm(r):
        q = q + s
        r = r - s*d
    return q, r

def mod_gauss(a, b): 
    q,r = divide_gaussian_integers(a, b)
    return r
a = (11 + 2*I) * (3 - 2*I)
b = (11 + 2*I) * (-5 +I)

euclides(a, b, mod_gauss)

11*I - 2

# 2. Teorema chino del resto

In [7]:
def chineseRemainder(coprimes, elems, div_mod, rem):
    m = coprimes[0]
    for i in range(1, len(coprimes)):
        m = m * coprimes[i]
    result = 0;
    for i in range(0, len(coprimes)):
        n = m / coprimes[i]
        _, a, _ = extendedEuclides(n, coprimes[i], div_mod)
        c = rem((a * elems[i]), coprimes[i])
        result = c * n + result
    return result

## 2.1 Ejemplos en $\mathbb{Z}$

In [8]:
def rem(a, b):
    r = abs(a) % abs(b)
    return r if a >= 0 else -r
chineseRemainder([3,5,7],[4,5,33], div_mod_int, rem)

40

## 2.2 Ejemplos en $\mathbb{Q}[x]$

In [9]:
R.<x> = QQ[]
a = (x + 1)
b =  (x^2 -2*x + 1)
print(extendedEuclides(a,b,div_mod_pol))
chineseRemainder([a,b],[x-2,x^2-4*x+4], div_mod_pol, mod_pol)

(4, -x + 3, 1)


0

# 3. MCD en DFU

In [10]:
def gcdDFU(a, b):
    if a.degree() < b.degree():
        return mcdDFU(b,a)
    while b != 0:
        q,r = (b.leading_coefficient()**(a.degree() - b.degree() + 1) * a).quo_rem(b)
        a = b
        b = r
    return a

## 3.1 Ejemplos en $\mathbb{Z}[x]$

In [11]:
R.<x> = ZZ[]
a = (x + 1)**2
b = (x + 1)*(x - 1)
gcdDFU(a, b).monic()

x + 1

# 4. Inverso de un elemento en un cuerpo finito

In [12]:
def inverseFiniteField(a, field, div_mod):
    _, _, b = extendedEuclides(field.cardinality(), a, div_mod)
    return field(b)

## 4.1 Ejemplos en $\mathbb{Z}_p$

In [13]:
Z7 = Integers(7)
a = 3
print(inverseFiniteField(a, Z7, div_mod_int))

Z31 = Integers(31)
b = 14
print(inverseFiniteField(b, Z31, div_mod_int))

Z61 = Integers(61)
c = 27
print(inverseFiniteField(c, Z61, div_mod_int))

5
20
52


# 5. Test de irreducibilidad de un polinomio en $\mathbb{F}_q[x]$

In [14]:
def fast_exp(x, e):
#Devuelve x^e
    if e==0: 
        return 1
    elif e==1: 
        return x
    elif e%2==0: 
        return fast_exp(x*x,e//2)
    else: 
        return x*fast_exp(x*x,e//2)

In [15]:
def fast_exp_mod(b,e,m):
#Devuelve b^e mod m.
    r = 1
    b = b%m
    while e>0:
        if e%2 == 1:
            r = (r*b)%m
        e = e//2
        b  = (b*b)%m
    return r 

In [16]:
def irreducibility_test(f,q):
    n = f.degree()
    F= factor(n)
    primes =[f[0] for f in F for i in range(f[1])]
    
    s = fast_exp((x),q)
    print s
    a = fast_exp_mod(s,n,f)
    print a
    if a != x: return False
    
    for k in primes:
        h = fast_exp_mod(s,n//t,f)
        if gcd(h-x, f) != 1: return False
    return True

In [17]:
R.<x>=PolynomialRing(Integers(14))
f = x^21+x^2+1
print "El polinomio ", f, " es irreducible en F14[x]?: ", irreducibility_test(f,14), " ", f.is_irreducible()
g = R.random_element(2)
print "El polinomio ", g, " es irreducible en F14[x]?: ", irreducibility_test(g,14), " ", g.is_irreducible()  	

El polinomio  x^21 + x^2 + 1  es irreducible en F14[x]?:  x^14


TypeError: no common canonical parent for objects with parents: 'Univariate Polynomial Ring in x over Ring of integers modulo 14' and '<type 'tuple'>'

In [ ]:
F= factor(2016)
primes =[f[0] for f in F for i in range(f[1])]
for k in primes:
    print k